# Importando Bibliotecas

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install jedi
!pip install -U spacy -q
!pip install -U pip setuptools wheel -q
!python -m spacy download pt_core_news_sm -q

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 41.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import os
import re
import nltk
import spacy
import string 
import pandas as pd

from copy import Error
from enum import Enum, auto
from spacy.tokens import Token
from google.colab import runtime
from nltk.stem import RSLPStemmer
from nltk.corpus import stopwords
from abc import ABC, abstractmethod
from typing import List, Type, Union 
from spacy.tokenizer import Tokenizer
from spacy.lang.xx import MultiLanguage
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from multiprocessing import Pool, cpu_count

In [4]:
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Pré-Processamento

## Definição da Pipeline

Criação de uma classe abstrata para as etapas poderem usar como interface durante a sua instanciação, foi criado está interface para garantir a manutenção e legibilidade do código.

Outrora, a classe "PreprocessingTextPipeline" será usada para instanciação para a criação da pipeline em si, permitindo adicionar etapas e garantir a execução.

Portanto, vale lembrar que existe duas opções de bibliotecas usadas, no caso são NLTK e Spacy.

In [5]:
class NLPLibrary(Enum):
    """
    Enum for supported NLP libraries.
    """
    nltk = 0
    spacy = 1

class TextProcessingStep(ABC):
    """
    Base class for a text processing step.
    """

    @abstractmethod
    def execute(self, data: Union[str, List[str]]) -> Union[str, List[str]]:
        """
        Executes the processing step.
        """
        pass

    @abstractmethod
    def requires(self) -> List[Type['TextProcessingStep']]:
        """
        Specifies steps that must be executed before this one.
        """
        pass

class PreprocessingTextPipeline:
    """
    Pipeline for managing and executing text preprocessing steps.
    """

    def __init__(self, nlpLibrary: NLPLibrary):
        """
        Initializes the pipeline.
        """
        self.steps = []
        self.nlpLibrary = nlpLibrary

    def add_step(self, step: TextProcessingStep):
        """
        Adds a step to the pipeline, checking dependencies.
        """
        for required_step in step.requires():
            if not any(isinstance(s, required_step) for s in self.steps):
                raise ValueError(f"{step.__class__.__name__} requires {required_step.__name__} to be added first.")

        self.steps.append(step)

    def run(self, data: Union[str, List[str]]) -> Union[str, List[str]]:
        """
        Executes all steps in the pipeline.
        """
        for step in self.steps:
            data = step.execute(data, self.nlpLibrary)
        return data


## Definição das etapas da Pipeline

Descrição das etapas e seus propósitos:

**Limpeza dos Dados**
   - **RemoveEscapeSequences**: Esta etapa remove sequências de escape, como quebras de linha, tabulações, eliminando caracteres desnecessários que não contribuem para o significado semântico do texto.
   - **RemoveNumbers**: Esta classe remove dígitos numéricos do texto usando expressões regulares, limpando os dados para focar no conteúdo textual.
   - **RemovePunctuation**: Esta etapa elimina sinais de pontuação, que geralmente não têm valor semântico significativo e podem introduzir ruído no processamento do texto.

**Normalização**
   - **Lowercase**: Converte todos os caracteres do texto para letras minúsculas, garantindo uniformidade e prevenindo discrepâncias devido à capitalização. Essa normalização é crucial para manter a consistência na representação das palavras.

**Tokenização**
   - **Tokenization**: O texto é segmentado em unidades menores chamadas tokens, que podem ser palavras ou frases. Essa segmentação permite que o modelo de NLP processe o texto de maneira mais granular, facilitando a análise subsequente, como stemming ou lematização.

**Remoção de Stop Words**
   - **RemoveStopwords**: Palavras comuns que não agregam muito valor semântico, como "e", "ou" e "é", são removidas. Esta etapa reduz a dimensionalidade do texto e enfatiza palavras mais significativas, melhorando a eficiência computacional e a precisão do modelo.

**Stemming ou Lemmatização**
   - **Stemming**: Esta etapa reduz as palavras às suas formas raiz. Por exemplo, "correndo", "corredor" e "correu" seriam todos reduzidos a "corr". Isso ajuda o modelo a reconhecer diferentes formas de uma palavra como o mesmo conceito.
   - **Lemmatization**: Similar ao stemming, mas mais sofisticada, a lematização reduz as palavras à sua forma base ou de dicionário, considerando o contexto e garantindo que a palavra mantenha seu significado.

**Referências:**

https://spotintelligence.com/2022/12/21/nltk-preprocessing-pipeline/

https://www.nltk.org/

https://spacy.io/api/doc




In [6]:
class RemoveEscapeSequences(TextProcessingStep):
    """
    Removes escape sequences from the text.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        return data.translate(str.maketrans({'\n': ' ', '\t': ' ', '\r': ' '}))

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class Lowercase(TextProcessingStep):
    """
    Converts all text to lowercase.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        return data.lower()

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class RemoveNumbers(TextProcessingStep):
    """
    Removes numbers from the text.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        return re.sub(r'\d+', '', data)

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class RemovePunctuation(TextProcessingStep):
    """
    Removes punctuation from the text.
    """
    def execute(self, data: str, library: NLPLibrary) -> str:
        translator = str.maketrans('', '', string.punctuation)
        return data.translate(translator)

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class Tokenization(TextProcessingStep):
    """
    Tokenizes the text into words.
    """
    def execute(self, data: str, library: NLPLibrary) -> List[str]:
        if library == NLPLibrary.nltk:
            return word_tokenize(data)
        elif library == NLPLibrary.spacy:
            tokens = MultiLanguage().tokenizer(data)
            return [token.text for token in tokens]

    def requires(self) -> List[Type[TextProcessingStep]]:
        return []

class RemoveStopwords(TextProcessingStep):
    """
    Removes stopwords from the text.
    """
    def execute(self, data: List[str], library: NLPLibrary) -> List[str]:
        if library == NLPLibrary.nltk:
            stop_words = set(stopwords.words('portuguese'))
            return [word for word in data if word.lower() not in stop_words]
        elif library == NLPLibrary.spacy:
            nlp = spacy.load("pt_core_news_sm")
            doc = nlp.make_doc(" ".join(data))
            return [token.text for token in doc if not token.is_stop]

    def requires(self) -> List[Type[TextProcessingStep]]:
        return [Tokenization]

class Stemming(TextProcessingStep):
    """
    Applies stemming to the text.
    """
    def execute(self, data: List[str], library: NLPLibrary) -> List[str]:
        if library == NLPLibrary.nltk:
            stemmer = RSLPStemmer()
            return [stemmer.stem(word) for word in data]
        elif library == NLPLibrary.spacy:
            raise Error("spacy does not provide built-in stemming resources")

    def requires(self) -> List[Type[TextProcessingStep]]:
        return [Tokenization, RemoveStopwords]

class Lemmatization(TextProcessingStep):
    """
    Applies lemmatization to the text.
    """
    def execute(self, data: List[str], library: NLPLibrary) -> List[str]:
        if library == NLPLibrary.nltk:
            raise Error("nltk does not provide built-in pt-br lemmatizer resources")
        elif library == NLPLibrary.spacy:
            nlp = spacy.load("pt_core_news_sm")
            doc = nlp(" ".join(data))
            return [token.lemma_ for token in doc]

    def requires(self) -> List[Type[TextProcessingStep]]:
        return [Tokenization, RemoveStopwords]

## Definição da sequência na Pipeline

In [7]:
# Build the sequence that belongs to Pipeline
pipeline = PreprocessingTextPipeline(NLPLibrary.spacy)

pipeline.add_step(RemoveEscapeSequences())
pipeline.add_step(Lowercase())
pipeline.add_step(RemoveNumbers())
pipeline.add_step(RemovePunctuation())
pipeline.add_step(Tokenization())
pipeline.add_step(RemoveStopwords())
pipeline.add_step(Lemmatization())

## Processamento da base de dados

Nesta seção, é mostrado o processamento da base de dados ao longo da pipeline. No final, uma pequena demonstração da base de dados processada num dataframe.

Observação: Na variável "path_database" substitua o valor desta pelo caminho que se encontra a base de dados no seu dispositivo.

In [8]:
def process_question(question):
    # Process a question using the NLP pipeline and join the results with commas.
    return ','.join(pipeline.run(question))

path_database = "/content/drive/MyDrive/data.xlsx"  # Path to the Excel file.

df = pd.read_excel(path_database)  # Load the Excel file into a DataFrame.

df.columns = ['no', 'intention', 'question', 'answers']  # Rename the DataFrame columns.

df = df.drop(columns='answers')  # Drop the 'answers' column as it's not needed.

with Pool(cpu_count()) as pool:
    # Process each question in parallel using the CPU cores.
    df['processed_question'] = pool.map(process_question, df['question'])

processed_df = df[['intention', 'question', 'processed_question']]  # Select relevant columns.

In [9]:
# Load the head of dataframe for demonstration
processed_df.head()

,intention,question,processed_question
0,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"diatudo,bemeu,gostar,caixa,family,Mart,verde,e..."
1,Como fazer remessa,Como enviar dinheiro do Japão?,"enviar,dinheiro,Japão"
2,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"levar,beneficiário,receber,dinheiro"
3,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,"acabar,transferência, ,total,Yenes"
4,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,"poder,remesso,yenes,bbb,rrr, ,yenes,Aaaa,Mm..."


In [10]:
# Export data for the excel format
processed_df.to_excel('processed_data.xlsx', index=False)

# Testes

In [11]:
import unittest # Library for doing unit and integrate tests

A partir da construção da pipeline, para a construção dos testes foi usado a biblioteca "unittest" e a frase abaixo , retirada da base de dados:

**"Bom dia, quanto da 150.000 yenes no Brasil hj ?"**

Para os testes, foram feitas duas seções, visto que a primeira se trata dos testes unitários e a segunda dos dois testes de integração.

### Testes Unitários

As funções dos testes unitários para serem testados e seus resultados esperados, estão logo abaixo:

- TestRemoveEscapeSequences (remove sequências de escape):
"Bom dia, quanto da 150.000 yenes no Brasil hj ?" -> "Bom dia, quanto da 150.000 yenes no Brasil hj ?"

- TestLowercase (sentença em caixa baixa):

"Bom dia, quanto da 150.000 yenes no Brasil hj ?" -> "bom dia, quanto da 150.000 yenes no brasil hj ?"

- TestRemoveNumbers (Remoção de números na sentença):

"Bom dia, quanto da 150.000 yenes no Brasil hj ?" -> "Bom dia, quanto da . yenes no Brasil hj ?"

- TestRemovePunctuation (Remoção de sinais de pontuação):

"Bom dia, quanto da 150.000 yenes no Brasil hj ?" -> "Bom dia quanto da 150000 yenes no Brasil hj"

- TestTokenization (Segmentação do texto em unidades menores chamadas tokens) :

"Bom dia, quanto da 150.000 yenes no Brasil hj ?" -> ['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?']

- TestRemoveStopwords (Palavras que são removidas do texto para reduzir a dimensionalidade e focar nas palavras de maior relevância):

['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?'] -> ['Bom', 'dia', ',', 'quanto', '150.000', 'yenes', 'Brasil', 'hj', '?']

- TestStemming (Conversão de palavras para seu radical base):

['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?'] -> ['Bom', 'dia', ',', 'quant', 'da', '150.000', 'yen', 'no', 'Brasil', 'hj', '?']

- TestLemmatization (Conversão de palavras para seus lemas):

['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?'] -> ['Bom', 'dia', ',', 'quanto', 'de o', '150.000', 'yene', 'em o', 'Brasil', 'hj', '?']

In [12]:
class TestRemoveEscapeSequences(unittest.TestCase):
    """
    Unit test for the removal of escape sequences from text.
    """

    def test_remove_escape_sequences(self):
        """
        Tests whether the `execute` method of the `RemoveEscapeSequences` class correctly removes escape sequences.
        """
        step = RemoveEscapeSequences()  # Instantiate the RemoveEscapeSequences class
        self.assertEqual(step.execute("Bom dia, quanto da 150.000 yenes no Brasil hj ?", NLPLibrary.nltk),
                         "Bom dia, quanto da 150.000 yenes no Brasil hj ?")

class TestLowercase(unittest.TestCase):
    """
    Unit test for the text conversion to lowercase step.
    """

    def unit_test_lowercase(self):
        """
        Tests whether the `execute` method of the `Lowercase` class correctly converts text to lowercase.
        """
        step = Lowercase()  # Instantiate the Lowercase class
        self.assertEqual(step.execute("Bom dia, quanto da 150.000 yenes no Brasil hj ?", NLPLibrary.nltk),
                         "bom dia, quanto da 150.000 yenes no brasil hj ?")


class TestTokenization(unittest.TestCase):
    """
    Unit test for the text tokenization step.
    """

    def unit_test_tokenization(self):
        """
        Tests whether the `execute` method of the `Tokenization` class correctly splits the text into tokens.
        """
        step = Tokenization()  # Instantiate the Tokenization class
        self.assertEqual(step.execute("Bom dia, quanto da 150.000 yenes no Brasil hj ?", NLPLibrary.nltk),
                         ['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?'])

class TestRemoveNumbers(unittest.TestCase):
    """
    Unit test for the text numbers removal step.
    """

    def unit_test_remove_numbers(self):
        """
        Tests whether the `execute` method of the `RemoveNumbers` class correctly removes numbers from the text.
        """
        step = RemoveNumbers()  # Instantiate the RemoveNumbers class
        self.assertEqual(step.execute("Bom dia, quanto da 150.000 yenes no Brasil hj ?", NLPLibrary.nltk),
                         "Bom dia, quanto da . yenes no Brasil hj ?")


class TestRemovePunctuation(unittest.TestCase):
    """
    Unit test for the text punctuation removal step.
    """

    def unit_test_remove_punctuation(self):
        """
        Tests whether the `execute` method of the `RemovePunctuation` class correctly removes punctuation from the text.
        """
        step = RemovePunctuation()  # Instantiate the RemovePunctuation class
        self.assertEqual(step.execute("Bom dia, quanto da 150.000 yenes no Brasil hj ?", NLPLibrary.nltk),
                         "Bom dia quanto da 150000 yenes no Brasil hj")

class TestRemoveStopwords(unittest.TestCase):
    """
    Unit test for the text stopwords removal step.
    """

    def unit_test_remove_stopwords(self):
        """
        Tests whether the `execute` method of the `RemoveStopwords` class correctly removes stopwords from the tokenized text.
        """
        step = RemoveStopwords()  # Instantiate the RemoveStopwords class
        self.assertEqual(step.execute(['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?'], NLPLibrary.nltk),
                         ['Bom', 'dia', ',', 'quanto', '150.000', 'yenes', 'Brasil', 'hj', '?'])


class TestStemming(unittest.TestCase):
    """
    Unit test for the text stemming step.
    """

    def unit_test_stemming(self):
        """
        Tests whether the `execute` method of the `Stemming` class correctly applies stemming to the tokenized text.
        """
        step = Stemming()  # Instantiate the Stemming class
        self.assertEqual(step.execute(['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?'], NLPLibrary.nltk),
                         ['Bom', 'dia', ',', 'quant', 'da', '150.000', 'yen', 'no', 'Brasil', 'hj', '?'])

class TestLemmatization(unittest.TestCase):
    """
    Unit test for the text lemmatization step.
    """

    def unit_test_lemmatization(self):
        """
        Tests whether the `execute` method of the `Lemmatization` class correctly applies lemmatization to the tokenized text.
        """
        step = Lemmatization()  # Instantiate the Lemmatization class
        self.assertEqual(step.execute(['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?'], NLPLibrary.spacy),
                         ['Bom', 'dia', ',', 'quanto', 'de o', '150.000', 'yene', 'em o', 'Brasil', 'hj', '?'])

### Resultados

A partir da execução dos testes, pode se concluir que todos os testes passaram comprovando a qualidade das etapas criadas para a pipeline.
Logo abaixo contém observações das mudanças ocorridas na frase dentre as funções definidas:

- RemoveEscapeSequences: Retirou as sequências de escape;

- Lowercase: A frase de teste ficou em caixa baixa, por exemplo, "Bom" ficou "bom";

- Remove Numbers: Ocorreu a retirada dos números, como por exemplo, "150.000" ficou ".";

- Remove Punctuation: Ocorreu a retirada de sinais de pontuação, como por exemplo, este sinal "?" foi retirado;

- Remove Stopwords: Ocorreu a remoção de palavras que não agregariam relevância ao significado da sentença, como por exemplo, o "no" foi retirado;

- Tokenization: Ocorreu de fato a separação da sentença em unidades de tokens, como por exemplo, "['Bom', 'dia', ',', 'quanto', 'da', '150.000', 'yenes', 'no', 'Brasil', 'hj', '?']";

- Stemming: Algumas palavras foram diminuidas para sua base, como por exemplo "quanto" ficou "quant";

- Lemmatization: Algumas palavras foram diminuidas, por exemplo "yenes" tirou o plural e ficou "yene";

In [13]:
# Execution of the tests
def run_unit_tests():
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestRemoveEscapeSequences))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestLowercase))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestTokenization))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestRemoveNumbers))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestRemovePunctuation))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestRemoveStopwords))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestStemming))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestLemmatization))

run_unit_tests()

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


### Testes Integrados

Foi feito dois testes integrados levando em consideração as duas possibilidades de bibliotecas na pipeline, primeiramente, foi feito com a biblioteca "NLKT". E por fim, o outro teste foi feito usando a biblioteca "Spacy".

Logo abaixo tem as transformações feitas que seguiram com exatidão todas as etapas:

- Pipeline usando o NLKT:

"Bom dia, quanto da 150.000 yenes no Brasil hj ?" -> ['bom', 'dia', 'quant', 'yen', 'brasil', 'hj']

- Pipeline usando o Spacy:

"Bom dia, quanto da 150.000 yenes no Brasil hj ?" -> ['dia', '   ', 'yenes', 'Brasil', 'hj']

In [14]:
class TestPreprocessingTextPipelineLibraryNLKT(unittest.TestCase):
    """
    Integration test for the PreprocessingTextPipeline using the NLKT NLP library.
    """

    def integration_test_pipeline(self):
        """
        Tests the entire preprocessing pipeline with the following steps:
        - Removing Escape Sequences
        - Lowercasing
        - Removing numbers
        - Removing punctuation
        - Tokenization
        - Removing stopwords
        - Stemming

        The test verifies that the pipeline processes the input text correctly using the NLKT library.
        """
        # Initialize the pipeline with the NLKT library
        pipeline = PreprocessingTextPipeline(NLPLibrary.nlkt)

        # Define each step in the preprocessing pipeline
        remove_escape_step = RemoveEscapeSequences()
        lowercase_step = Lowercase()
        remove_numbers_step = RemoveNumbers()
        remove_punctuation_step = RemovePunctuation()
        tokenization_step = Tokenization()
        remove_stopwords_step = RemoveStopwords()
        stemming_step = Stemming()
        lemmatization_step = Lemmatization()

        # Add steps to the pipeline
        pipeline.add_step(remove_escape_step)
        pipeline.add_step(lowercase_step)
        pipeline.add_step(remove_numbers_step)
        pipeline.add_step(remove_punctuation_step)
        pipeline.add_step(tokenization_step)
        pipeline.add_step(remove_stopwords_step)
        pipeline.add_step(stemming_step)

        # Run the pipeline on the input text
        result = pipeline.run("Bom dia, quanto da 150.000 yenes no Brasil hj ?")
        expected_result = ['bom', 'dia', 'quant', 'yen', 'brasil', 'hj']

        # Assert that the result matches the expected output
        self.assertEqual(result, expected_result)

class TestPreprocessingTextPipelineLibrarySpacy(unittest.TestCase):
    """
    Integration test for the PreprocessingTextPipeline using the Spacy NLP library.
    """

    def integration_test_pipeline(self):
        """
        Tests the entire preprocessing pipeline with the following steps:
        - Removing Escape Sequences
        - Lowercasing
        - Removing numbers
        - Removing punctuation
        - Tokenization
        - Removing stopwords
        - Lemmatization

        The test verifies that the pipeline processes the input text correctly using the Spacy library.
        """
        # Initialize the pipeline with the Spacy library
        pipeline_spacy = PreprocessingTextPipeline(NLPLibrary.spacy)

        # Define each step in the preprocessing pipeline
        remove_escape_step = RemoveEscapeSequences()
        lowercase_step = Lowercase()
        remove_numbers_step = RemoveNumbers()
        remove_punctuation_step = RemovePunctuation()
        tokenization_step = Tokenization()
        remove_stopwords_step = RemoveStopwords()
        stemming_step = Stemming()
        lemmatization_step = Lemmatization()

        # Add steps to the pipeline
        pipeline_spacy.add_step(remove_escape_step)
        pipeline_spacy.add_step(lowercase_step)
        pipeline_spacy.add_step(remove_numbers_step)
        pipeline_spacy.add_step(remove_punctuation_step)
        pipeline_spacy.add_step(tokenization_step)
        pipeline_spacy.add_step(remove_stopwords_step)
        pipeline_spacy.add_step(lemmatization_step)

        # Run the pipeline on the input text
        result = pipeline_spacy.run("Bom dia, quanto da 150.000 yenes no Brasil hj ?")
        expected_result = ['dia', '   ', 'yenes', 'Brasil', 'hj']

        # Assert that the result matches the expected output
        self.assertEqual(result, expected_result)


### Resultados

Ambos os testes passaram demonstrando qualidade da pipeline em si.

In [15]:
# Execution of the tests
def run_integrate_tests():
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestPreprocessingTextPipelineLibraryNLKT))
    unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestPreprocessingTextPipelineLibrarySpacy))

run_integrate_tests()


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
